<a href="https://colab.research.google.com/github/tunaemirkandemir/xdeepfm_ctr/blob/main/xdeepfm_criteo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deepctr-torch

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
from deepctr_torch.models import xDeepFM
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names

In [ ]:
URL = 'https://raw.githubusercontent.com/shenweichen/DeepCTR/master/examples/criteo_sample.txt'
df = pd.read_csv(URL)

In [ ]:
df

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,NaN,3,260.0,NaN,17668.0,NaN,NaN,33.0,NaN,...,e5ba7672,87c6f83c,NaN,NaN,0429f84b,NaN,3a171ecb,c0d61a5c,NaN,NaN
1,0,NaN,-1,19.0,35.0,30251.0,247.0,1.0,35.0,160.0,...,d4bb7bd8,6fc84bfb,NaN,NaN,5155d8a3,NaN,be7c41b4,ded4aac9,NaN,NaN
2,0,0.0,0,2.0,12.0,2013.0,164.0,6.0,35.0,523.0,...,e5ba7672,675c9258,NaN,NaN,2e01979f,NaN,bcdee96c,6d5d1302,NaN,NaN
3,0,NaN,13,1.0,4.0,16836.0,200.0,5.0,4.0,29.0,...,e5ba7672,52e44668,NaN,NaN,e587c466,NaN,32c7478e,3b183c5c,NaN,NaN
4,0,0.0,0,104.0,27.0,1990.0,142.0,4.0,32.0,37.0,...,e5ba7672,25c88e42,21ddcdc9,b1252a9d,0e8585d2,NaN,32c7478e,0d4a6d1a,001f3601,92c878de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,NaN,0,113.0,3.0,3036.0,575.0,2.0,3.0,214.0,...,07c540c4,9880032b,21ddcdc9,5840adea,34cc61bb,c9d4222a,32c7478e,e5ed7da2,ea9a246c,984e0db0
196,1,0.0,1,1.0,1.0,1607.0,12.0,1.0,12.0,15.0,...,1e88c74f,3972b4ed,NaN,NaN,d1aa4512,NaN,32c7478e,9257f75f,NaN,NaN
197,1,1.0,0,6.0,3.0,0.0,0.0,19.0,3.0,3.0,...,3486227d,5aed7436,54591762,a458ea53,4a2c3526,NaN,32c7478e,1793a828,e8b83407,1a02cbe1
198,0,0.0,22,6.0,22.0,203.0,153.0,80.0,18.0,508.0,...,3486227d,13145934,55dd3565,5840adea,bf647035,NaN,32c7478e,1481ceb4,e8b83407,988b0775


In [ ]:
df.columns


Index(['label', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10',
       'I11', 'I12', 'I13', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8',
       'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18',
       'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26'],
      dtype='object')

In [ ]:
dfx = df.copy()

In [ ]:
target = ['label']

In [ ]:
dense_columns = []
sparse_columns = []

for dense_col in range(1,14):
  dense_columns.append( dfx.columns[dense_col])

for sparse_col in range(14,len(dfx.columns)):
  sparse_columns.append( dfx.columns[sparse_col])


In [ ]:
dense_columns

['I1',
 'I2',
 'I3',
 'I4',
 'I5',
 'I6',
 'I7',
 'I8',
 'I9',
 'I10',
 'I11',
 'I12',
 'I13']

In [ ]:
for dense_col in dense_columns:
  dfx[dense_col] = dfx[dense_col].fillna('0',)

In [ ]:
dfx

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,0,3,260.0,0,17668.0,0,0,33.0,0,...,e5ba7672,87c6f83c,-1,-1,0429f84b,-1,3a171ecb,c0d61a5c,-1,-1
1,0,0,-1,19.0,35.0,30251.0,247.0,1.0,35.0,160.0,...,d4bb7bd8,6fc84bfb,-1,-1,5155d8a3,-1,be7c41b4,ded4aac9,-1,-1
2,0,0.0,0,2.0,12.0,2013.0,164.0,6.0,35.0,523.0,...,e5ba7672,675c9258,-1,-1,2e01979f,-1,bcdee96c,6d5d1302,-1,-1
3,0,0,13,1.0,4.0,16836.0,200.0,5.0,4.0,29.0,...,e5ba7672,52e44668,-1,-1,e587c466,-1,32c7478e,3b183c5c,-1,-1
4,0,0.0,0,104.0,27.0,1990.0,142.0,4.0,32.0,37.0,...,e5ba7672,25c88e42,21ddcdc9,b1252a9d,0e8585d2,-1,32c7478e,0d4a6d1a,001f3601,92c878de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0,0,113.0,3.0,3036.0,575.0,2.0,3.0,214.0,...,07c540c4,9880032b,21ddcdc9,5840adea,34cc61bb,c9d4222a,32c7478e,e5ed7da2,ea9a246c,984e0db0
196,1,0.0,1,1.0,1.0,1607.0,12.0,1.0,12.0,15.0,...,1e88c74f,3972b4ed,-1,-1,d1aa4512,-1,32c7478e,9257f75f,-1,-1
197,1,1.0,0,6.0,3.0,0.0,0.0,19.0,3.0,3.0,...,3486227d,5aed7436,54591762,a458ea53,4a2c3526,-1,32c7478e,1793a828,e8b83407,1a02cbe1
198,0,0.0,22,6.0,22.0,203.0,153.0,80.0,18.0,508.0,...,3486227d,13145934,55dd3565,5840adea,bf647035,-1,32c7478e,1481ceb4,e8b83407,988b0775


In [ ]:
for sparse_col in sparse_columns:
  dfx[sparse_col] = dfx[sparse_col].fillna('-1',)

In [ ]:
dfx

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,0,3,260.0,0,17668.0,0,0,33.0,0,...,e5ba7672,87c6f83c,-1,-1,0429f84b,-1,3a171ecb,c0d61a5c,-1,-1
1,0,0,-1,19.0,35.0,30251.0,247.0,1.0,35.0,160.0,...,d4bb7bd8,6fc84bfb,-1,-1,5155d8a3,-1,be7c41b4,ded4aac9,-1,-1
2,0,0.0,0,2.0,12.0,2013.0,164.0,6.0,35.0,523.0,...,e5ba7672,675c9258,-1,-1,2e01979f,-1,bcdee96c,6d5d1302,-1,-1
3,0,0,13,1.0,4.0,16836.0,200.0,5.0,4.0,29.0,...,e5ba7672,52e44668,-1,-1,e587c466,-1,32c7478e,3b183c5c,-1,-1
4,0,0.0,0,104.0,27.0,1990.0,142.0,4.0,32.0,37.0,...,e5ba7672,25c88e42,21ddcdc9,b1252a9d,0e8585d2,-1,32c7478e,0d4a6d1a,001f3601,92c878de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0,0,113.0,3.0,3036.0,575.0,2.0,3.0,214.0,...,07c540c4,9880032b,21ddcdc9,5840adea,34cc61bb,c9d4222a,32c7478e,e5ed7da2,ea9a246c,984e0db0
196,1,0.0,1,1.0,1.0,1607.0,12.0,1.0,12.0,15.0,...,1e88c74f,3972b4ed,-1,-1,d1aa4512,-1,32c7478e,9257f75f,-1,-1
197,1,1.0,0,6.0,3.0,0.0,0.0,19.0,3.0,3.0,...,3486227d,5aed7436,54591762,a458ea53,4a2c3526,-1,32c7478e,1793a828,e8b83407,1a02cbe1
198,0,0.0,22,6.0,22.0,203.0,153.0,80.0,18.0,508.0,...,3486227d,13145934,55dd3565,5840adea,bf647035,-1,32c7478e,1481ceb4,e8b83407,988b0775


In [ ]:
dfx2 = dfx.copy()

In [ ]:
label_encoder = LabelEncoder()
for col in sparse_columns:
  dfx2[col] = label_encoder.fit_transform(dfx2[col])

In [ ]:
scaler = MinMaxScaler(feature_range = (0,1))

In [ ]:

dfx2[dense_columns] = scaler.fit_transform(dfx2[dense_columns])

In [ ]:
dfx2

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,0.000000,0.001332,0.092362,0.000000,0.034825,0.000000,0.000000,0.673469,0.000000,...,8,66,0,0,3,0,1,96,0,0
1,0,0.000000,0.000000,0.006750,0.402299,0.059628,0.117284,0.003322,0.714286,0.154739,...,7,52,0,0,47,0,7,112,0,0
2,0,0.000000,0.000333,0.000710,0.137931,0.003968,0.077873,0.019934,0.714286,0.505803,...,8,49,0,0,25,0,6,53,0,0
3,0,0.000000,0.004664,0.000355,0.045977,0.033185,0.094967,0.016611,0.081633,0.028046,...,8,37,0,0,156,0,0,32,0,0
4,0,0.000000,0.000333,0.036945,0.310345,0.003922,0.067426,0.013289,0.653061,0.035783,...,8,14,5,3,9,0,0,5,1,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0.000000,0.000333,0.040142,0.034483,0.005984,0.273029,0.006645,0.061224,0.206963,...,0,74,5,1,30,5,0,118,17,48
196,1,0.000000,0.000666,0.000355,0.011494,0.003168,0.005698,0.003322,0.244898,0.014507,...,1,25,0,0,138,0,0,68,0,0
197,1,0.027027,0.000333,0.002131,0.034483,0.000000,0.000000,0.063123,0.061224,0.002901,...,4,40,17,2,41,0,0,12,16,11
198,0,0.000000,0.007662,0.002131,0.252874,0.000400,0.072650,0.265781,0.367347,0.491296,...,4,7,18,1,123,0,0,10,16,49


In [ ]:
sparse_embeddings = [SparseFeat( sparse_col , vocabulary_size=dfx2[sparse_col].nunique(), embedding_dim = 4) for sparse_col in sparse_columns]

In [ ]:
sparse_embeddings

[SparseFeat(name='C1', vocabulary_size=27, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C1', group_name='default_group'),
 SparseFeat(name='C2', vocabulary_size=92, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C2', group_name='default_group'),
 SparseFeat(name='C3', vocabulary_size=172, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C3', group_name='default_group'),
 SparseFeat(name='C4', vocabulary_size=157, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C4', group_name='default_group'),
 SparseFeat(name='C5', vocabulary_size=12, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C5', group_name='default_group'),
 SparseFeat(name='C6', vocabulary_size=7, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C6', group_name='default_group'),
 SparseFeat(name='C7', vocabulary_size=183, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C7', group_name='default_group'),
 SparseFeat

In [ ]:
dense_embeddings = [DenseFeat(col , 1) for col in dense_columns]

In [ ]:
dense_embeddings

[DenseFeat(name='I1', dimension=1, dtype='float32'),
 DenseFeat(name='I2', dimension=1, dtype='float32'),
 DenseFeat(name='I3', dimension=1, dtype='float32'),
 DenseFeat(name='I4', dimension=1, dtype='float32'),
 DenseFeat(name='I5', dimension=1, dtype='float32'),
 DenseFeat(name='I6', dimension=1, dtype='float32'),
 DenseFeat(name='I7', dimension=1, dtype='float32'),
 DenseFeat(name='I8', dimension=1, dtype='float32'),
 DenseFeat(name='I9', dimension=1, dtype='float32'),
 DenseFeat(name='I10', dimension=1, dtype='float32'),
 DenseFeat(name='I11', dimension=1, dtype='float32'),
 DenseFeat(name='I12', dimension=1, dtype='float32'),
 DenseFeat(name='I13', dimension=1, dtype='float32')]

In [ ]:
dnn_embeddings = sparse_embeddings + dense_embeddings
linear_embeddings = sparse_embeddings + dense_embeddings
feature_names = get_feature_names(linear_embeddings + dnn_embeddings)




In [ ]:
feature_names

In [ ]:
train , test = train_test_split(dfx2 , test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

In [ ]:
xdeepfm_model =  xDeepFM(linear_embeddings , dnn_embeddings ,
                         dnn_hidden_units=(256, 256), cin_layer_size=(256, 128),
                         cin_split_half=True, cin_activation='relu',
                         l2_reg_linear=1e-05, l2_reg_embedding=1e-05,
                         l2_reg_dnn=0, l2_reg_cin=0, init_std=0.0001,
                         dnn_dropout=0, dnn_activation='relu', dnn_use_bn=False, task='binary', device='cpu', gpus=None)

In [ ]:
xdeepfm_model.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'])

In [ ]:
history_logs = xdeepfm_model.fit(train_model_input, train[['label']].values,
                    batch_size=256, epochs=10, verbose=2, validation_split=0.2)

cpu
Train on 128 samples, validate on 32 samples, 1 steps per epoch
Epoch 1/10
0s - loss:  0.6942 - binary_crossentropy:  0.6942 - val_binary_crossentropy:  0.6775
Epoch 2/10
0s - loss:  0.6706 - binary_crossentropy:  0.6706 - val_binary_crossentropy:  0.6630
Epoch 3/10
0s - loss:  0.6490 - binary_crossentropy:  0.6490 - val_binary_crossentropy:  0.6499
Epoch 4/10
0s - loss:  0.6287 - binary_crossentropy:  0.6287 - val_binary_crossentropy:  0.6378
Epoch 5/10
0s - loss:  0.6095 - binary_crossentropy:  0.6095 - val_binary_crossentropy:  0.6269
Epoch 6/10
0s - loss:  0.5916 - binary_crossentropy:  0.5916 - val_binary_crossentropy:  0.6170
Epoch 7/10
0s - loss:  0.5746 - binary_crossentropy:  0.5746 - val_binary_crossentropy:  0.6081
Epoch 8/10
0s - loss:  0.5586 - binary_crossentropy:  0.5586 - val_binary_crossentropy:  0.6002
Epoch 9/10
0s - loss:  0.5434 - binary_crossentropy:  0.5434 - val_binary_crossentropy:  0.5931
Epoch 10/10
0s - loss:  0.5290 - binary_crossentropy:  0.5290 - val_

In [ ]:
history_logs

In [ ]:
predictions = xdeepfm_model.predict(test_model_input, batch_size=256)

In [ ]:
predictions

array([[0.35722047],
       [0.35748413],
       [0.34522015],
       [0.33892953],
       [0.32409599],
       [0.34261036],
       [0.34152833],
       [0.33821851],
       [0.34366903],
       [0.35056245],
       [0.35030305],
       [0.34702456],
       [0.34718239],
       [0.34420839],
       [0.33310637],
       [0.34807947],
       [0.34377715],
       [0.33166662],
       [0.34613425],
       [0.3586688 ],
       [0.33991444],
       [0.34709889],
       [0.34923008],
       [0.33665085],
       [0.3569198 ],
       [0.33143055],
       [0.33713341],
       [0.33710635],
       [0.34603602],
       [0.34364218],
       [0.35187051],
       [0.37490016],
       [0.33977973],
       [0.33883595],
       [0.33413687],
       [0.32648855],
       [0.34462857],
       [0.31504577],
       [0.34437102],
       [0.34420863]])

In [98]:
flattened_predictions = predictions.flatten()

# Get the indices that would sort the flattened array
sorted_indices = np.argsort(flattened_predictions)[::-1]

# Sort the original array using the sorted indices
sorted_ctrs = predictions[sorted_indices]

In [99]:
sorted_ctrs

array([[0.37490016],
       [0.3586688 ],
       [0.35748413],
       [0.35722047],
       [0.3569198 ],
       [0.35187051],
       [0.35056245],
       [0.35030305],
       [0.34923008],
       [0.34807947],
       [0.34718239],
       [0.34709889],
       [0.34702456],
       [0.34613425],
       [0.34603602],
       [0.34522015],
       [0.34462857],
       [0.34437102],
       [0.34420863],
       [0.34420839],
       [0.34377715],
       [0.34366903],
       [0.34364218],
       [0.34261036],
       [0.34152833],
       [0.33991444],
       [0.33977973],
       [0.33892953],
       [0.33883595],
       [0.33821851],
       [0.33713341],
       [0.33710635],
       [0.33665085],
       [0.33413687],
       [0.33310637],
       [0.33166662],
       [0.33143055],
       [0.32648855],
       [0.32409599],
       [0.31504577]])